## 4D dynunet pipeline with NeuroI ROI dataset
* Brain image + mask image -> 4D modalities
* in_channels: 4, out_channels:1

In [1]:
import logging
import os
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser

import torch
import torch.distributed as dist
from monai.config import print_config
from monai.handlers import (
    CheckpointSaver,
    LrScheduleHandler,
    MeanDice,
    StatsHandler,
    ValidationHandler,
    from_engine,
)
from monai.inferers import SimpleInferer, SlidingWindowInferer
from monai.losses import DiceCELoss
from monai.utils import set_determinism
from torch.nn.parallel import DistributedDataParallel

from create_dataset import get_data_ke
from create_network import get_network_ke
from evaluator import DynUNetEvaluator
# from task_params import data_loader_params, patch_size
from trainer import DynUNetTrainer

from monai.utils import first
import numpy as np


In [2]:
# task_id = "01"
# fold = 0
# root_dir = "/data/kehyeong/project/MONAI_examples/data/brats/"
# datalist_path ="config/"

config = "/data/kehyeong/project/MONAI_examples/dynunet_pipeline/config_roi_earlystop_toy_220209.yaml"
train_dataset = "/work/NeuroI-models/ke-monai/data/roi/dataset_train_roi_toy2.csv"
val_dataset = "/work/NeuroI-models/ke-monai/data/roi/dataset_val_roi_toy2.csv"
log_file = "/data/train/running/l/model_roi_toy_220210/train.log"
checkpoint = None

## [아래 config에서 불러 오는 params]
# max_epochs = 1500
# num_samples = 4
# train_num_workers = 4
# val_num_workers = 2

patch_size = [64, 64, 64] # [128, 128, 128]    #[96, 96, 96]
learning_rate = 1.0e-3    # working: 1e-1
interval = 2
multi_gpu = False  # True
local_rank = 0

## [Dafault setting] - 변경 필요 없음
window_mode = "gaussian"  # "constant", "gaussian"
eval_overlap = 0.5
tta_val = True
batch_dice = False
lr_decay_flag = False
spacing = [1.0, 1.0, 1.0]
deep_supr_num = 3
expr_name = "baseline"
####################################


# pos_sample_num = 1
# neg_sample_num = 1
# cache_rate = 1.0


# mode = "train"
# checkpoint = None
# amp = False
# lr_decay = False
# tta_val = True
# batch_dice = False
# determinism_flag = False
# determinism_seed = 0
# expr_name = "baseline"

In [3]:
from config import get_config
from dataset_roi_4d import get_train_loader, get_val_loader   # 4D modality test
import monai

local_rank = local_rank
log_file = log_file
train_dataset = train_dataset
val_dataset = val_dataset
checkpoint = checkpoint

multi_gpu_flag = multi_gpu
config = get_config(config)
data_dir = config["data_dir"]
image_file_path = config["image_file_path"]
label_file_path = config["label_file_path"]
# brain_file_path = config["brain_file_path"]
mask_file_path = config["mask_file_path"]
random_seed = config["random_seed"]
max_epochs = config["train"]["max_epoches"]
num_classes = config["num_classes"]

# patch_size = tuple(config["patch_size"])
lr = config["train"]["lr"]
train_batch_size = config["train"]["batch_size"]
train_num_samples = config["train"]["num_samples"]
train_num_workers = config["train"]["num_workers"]
# val_interval = config["train"]["val_interval"]
val_batch_size = config["val"]["batch_size"]
sw_batch_size = val_batch_size    # for evaluator
val_num_workers = config["val"]["num_workers"]
log_dir = config["log_dir"]
model_dir = config["model_dir"]
mlflow_dir = os.path.join(log_dir, "mlruns")

amp_flag = (True if monai.utils.get_torch_version_tuple() >= (1, 6) else False,)

monai.utils.set_determinism(random_seed)
np.random.seed(random_seed)

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5,6"

if multi_gpu_flag:
    dist.init_process_group(backend="nccl", init_method="env://")
    device = torch.device(f"cuda:{local_rank}")
    torch.cuda.set_device(device)
else:
#     device = torch.device("cpu")
    device = torch.device("cuda")


#
# data loader
#
train_loader = get_train_loader(
    data_dir=data_dir,
    id_file=train_dataset,
    image_file_pattern=image_file_path,
    label_file_pattern=label_file_path,
    # brain_file_pattern=brain_file_path,
    mask_file_pattern=mask_file_path,
    batch_size=train_batch_size,
    patch_size=patch_size,
    num_samples=train_num_samples,
    num_workers=train_num_workers,
    multi_gpu_flag=multi_gpu_flag
)
val_loader = get_val_loader(
    data_dir=data_dir,
    id_file=val_dataset,
    image_file_pattern=image_file_path,
    label_file_pattern=label_file_path,
    # brain_file_pattern=brain_file_path,
    mask_file_pattern=mask_file_path,
    batch_size=val_batch_size,
    num_workers=val_num_workers,
    multi_gpu_flag=multi_gpu_flag
)

{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0217 00_0
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0217 01_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0217 02_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0217 03_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0005 00_0
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0005 01_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0005 02_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0005 03_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0419 00_0
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0419 01_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0419 02_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0419 03_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmente

Loading dataset: 100%|█████████████████████████████████████████████████████████████████████████| 52/52 [00:20<00:00,  2.50it/s]


{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0454 00_0
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0454 01_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0303 00_0
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0303 01_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0197 00_0
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0197 01_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0350 00_0
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0350 01_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0317 00_0
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0317 01_d


Loading dataset: 100%|█████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.70it/s]


In [4]:
num_classes

109

In [5]:
test_data = first(train_loader)
print(f"배치 수: num_sample({train_num_samples}) x batch_num({train_batch_size}) -> {train_num_samples*train_batch_size}")
print(test_data.keys())
print("image shape:", test_data['image'].shape)
print("label shape:", test_data['label'].shape)
print("image dtype:", test_data['image'].dtype)
print("label dtype:", test_data['label'].dtype)
print("1번 배치의 유니크한 라벨 리스트:", np.unique(test_data['label']))
total_labels = np.unique(test_data['label'])
print(f'1번 배치의 유니크한 라벨 class 수: {len(total_labels)}')

배치 수: num_sample(4) x batch_num(3) -> 12
dict_keys(['image', 'label', 'mask', 'image_meta_dict', 'label_meta_dict', 'mask_meta_dict', 'image_transforms', 'label_transforms', 'mask_transforms', 'resample_flag', 'anisotrophy_flag'])
image shape: torch.Size([12, 2, 64, 64, 64])
label shape: torch.Size([12, 1, 64, 64, 64])
image dtype: torch.float32
label dtype: torch.uint8
1번 배치의 유니크한 라벨 리스트: [  0   1   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  20  21  23  25  26  27  28  29  30  31  32  33  34  36  37  39  40  41
  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59
  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77
  78  79  80  81  83  85  86  87  88  89  90  92  93  94  95  96  97  98
  99 100 101 102 103 104 105 106 107 108]
1번 배치의 유니크한 라벨 class 수: 100


In [6]:
# 배치 1번의 전체 라벨의 shape과 유니크한 라벨 수 조사.
for each in test_data['label']:
    print(each.shape)
    total_labels = np.unique(each)
    print(f'class 수 {len(total_labels)}')

torch.Size([1, 64, 64, 64])
class 수 18
torch.Size([1, 64, 64, 64])
class 수 35
torch.Size([1, 64, 64, 64])
class 수 40
torch.Size([1, 64, 64, 64])
class 수 19
torch.Size([1, 64, 64, 64])
class 수 35
torch.Size([1, 64, 64, 64])
class 수 44
torch.Size([1, 64, 64, 64])
class 수 25
torch.Size([1, 64, 64, 64])
class 수 29
torch.Size([1, 64, 64, 64])
class 수 49
torch.Size([1, 64, 64, 64])
class 수 33
torch.Size([1, 64, 64, 64])
class 수 24
torch.Size([1, 64, 64, 64])
class 수 67


In [7]:
properties = {
    'modality': [0,1],
    'labels': np.arange(num_classes)
}
n_class = len(properties["labels"])
in_channels = len(properties["modality"])
in_channels, n_class

(2, 109)

In [8]:
# produce the network
val_output_dir = "./runs_fold{}_{}/".format(1, expr_name)
checkpoint = checkpoint
net = get_network_ke(properties, patch_size, spacing, deep_supr_num, 
                     val_output_dir, checkpoint)
net = net.to(device)
print(net)

if multi_gpu_flag:
    net = DistributedDataParallel(module=net, device_ids=[device])
# net = DistributedDataParallel(module=net, device_ids=[device])

optimizer = torch.optim.SGD(
    net.parameters(),
    lr=learning_rate,
    momentum=0.99,
    weight_decay=3e-5,
    nesterov=True,
)

scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lr_lambda=lambda epoch: (1 - epoch / max_epochs) ** 0.9
)
# produce evaluator
val_handlers = [
    StatsHandler(output_transform=lambda x: None),
    CheckpointSaver(
        save_dir=val_output_dir, save_dict={"net": net}, save_key_metric=True
    ),
]

evaluator = DynUNetEvaluator(
    device=device,
    val_data_loader=val_loader,
    network=net,
    num_classes=len(properties["labels"]),
    inferer=SlidingWindowInferer(
        roi_size=patch_size,
        sw_batch_size=sw_batch_size,
        overlap=eval_overlap,
        mode=window_mode,
    ),
    postprocessing=None,
    key_val_metric={
        "val_mean_dice": MeanDice(
            include_background=False,
            output_transform=from_engine(["pred", "label"]),
        )
    },
    val_handlers=val_handlers,
    amp=amp_flag,
    tta_val=tta_val,
)

# produce trainer
loss = DiceCELoss(to_onehot_y=True, softmax=True, batch=batch_dice)
train_handlers = []
if lr_decay_flag:
    train_handlers += [LrScheduleHandler(lr_scheduler=scheduler, print_lr=True)]

train_handlers += [
    ValidationHandler(validator=evaluator, interval=interval, epoch_level=True),
    StatsHandler(
#         tag_name="train_loss", output_transform=from_engine(["loss"], first=True)
        tag_name="train_loss", output_transform=lambda x: x["loss"]
    ),
]

trainer = DynUNetTrainer(
    device=device,
    max_epochs=max_epochs,
    train_data_loader=train_loader,
    network=net,
    optimizer=optimizer,
    loss_function=loss,
    inferer=SimpleInferer(),
    postprocessing=None,
    key_train_metric=None,
    train_handlers=train_handlers,
    amp=amp_flag,
)

if local_rank > 0:
    evaluator.logger.setLevel(logging.WARNING)
    trainer.logger.setLevel(logging.WARNING)

logger = logging.getLogger()

formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

# Setup file handler
fhandler = logging.FileHandler(log_file)
fhandler.setLevel(logging.INFO)
fhandler.setFormatter(formatter)

logger.addHandler(fhandler)

chandler = logging.StreamHandler()
chandler.setLevel(logging.INFO)
chandler.setFormatter(formatter)
logger.addHandler(chandler)

logger.setLevel(logging.INFO)

trainer.run()

2022-02-17 00:59:13,366 - ignite.engine.engine.DynUNetTrainer - INFO - Engine run resuming from iteration 0, epoch 0 until 1500 epochs


DynUNet(
  (input_block): UnetBasicBlock(
    (conv1): Convolution(
      (conv): Conv3d(2, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    )
    (conv2): Convolution(
      (conv): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    )
    (lrelu): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm1): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (norm2): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  )
  (downsamples): ModuleList(
    (0): UnetBasicBlock(
      (conv1): Convolution(
        (conv): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      )
      (conv2): Convolution(
        (conv): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      )
      (lrelu): LeakyReLU(negative_slope=0.01, inplace=True)
      (norm1): InstanceNorm3d(64, eps=1e-05

2022-02-17 00:59:22,383 - ignite.engine.engine.DynUNetTrainer - INFO - Epoch: 1/1500, Iter: 1/17 -- train_loss: 11.3886 
2022-02-17 00:59:22,732 - ignite.engine.engine.DynUNetTrainer - INFO - Epoch: 1/1500, Iter: 2/17 -- train_loss: 11.3494 
2022-02-17 00:59:23,115 - ignite.engine.engine.DynUNetTrainer - INFO - Epoch: 1/1500, Iter: 3/17 -- train_loss: 11.3137 
2022-02-17 00:59:23,469 - ignite.engine.engine.DynUNetTrainer - INFO - Epoch: 1/1500, Iter: 4/17 -- train_loss: 11.2763 
2022-02-17 00:59:24,023 - ignite.engine.engine.DynUNetTrainer - INFO - Epoch: 1/1500, Iter: 5/17 -- train_loss: 11.1088 
2022-02-17 00:59:24,365 - ignite.engine.engine.DynUNetTrainer - INFO - Epoch: 1/1500, Iter: 6/17 -- train_loss: 10.9822 
2022-02-17 00:59:24,695 - ignite.engine.engine.DynUNetTrainer - INFO - Epoch: 1/1500, Iter: 7/17 -- train_loss: 10.9689 
2022-02-17 00:59:25,096 - ignite.engine.engine.DynUNetTrainer - INFO - Epoch: 1/1500, Iter: 8/17 -- train_loss: 10.8720 
2022-02-17 00:59:25,524 - ignite

KeyboardInterrupt: 

In [ ]:
test_data = first(train_loader)
print(test_data.keys())
print('image, label shape')
print(test_data['image'].shape)
print(test_data['label'].shape)
print(test_data['image'].dtype)
print(test_data['label'].dtype)
print(np.unique(test_data['label']))
print(len(np.unique(test_data['label'])))

In [ ]:
test_data['label'].numel(), 24*96*96*96

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
H=60

test_data = first(train_loader)
image, label = (test_data["image"][0][0], test_data["label"][0][0])
print(test_data['image'].shape, test_data['label'].shape)
print(f"image shape: {image.shape}, label shape: {label.shape}")
print(f"image dtype: {image.dtype}, label dtype: {label.dtype}")
# plot the slice [:, :, 80]
plt.figure("check", (15, 10))
plt.subplot(1, 2, 1)
plt.title("image")
plt.imshow(image[:, :, H], cmap="gray")
plt.subplot(1, 2, 2)
plt.title("label")
plt.imshow(label[:, :, H])
# plt.subplot(1, 4, 3)
# plt.title("brain")
# plt.imshow(brain[:, :, H], cmap="gray")
# plt.subplot(1, 4, 4)
# plt.title("mask")
# plt.imshow(mask[:, :, H])
plt.show()

print(np.unique(label))

In [ ]:
np.unique(label), label

In [ ]:
image

In [ ]:
pixel_min, pixel_max = image.min().item(), image.max().item()
print(pixel_min, pixel_max)
histogram, bin_edges = np.histogram(image, bins=256, range=(pixel_min, pixel_max))
plt.figure()
plt.title("Grayscale Histogram")
plt.xlabel("grayscale value")
plt.ylabel("pixel count")
plt.xlim([pixel_min, pixel_max])  # <- named arguments do not work here

plt.plot(bin_edges[0:-1], histogram)  # <- or here
plt.show()

In [ ]:
new_img = np.where(image < 0, 2, image)
new_img.shape
plt.figure("check", (15, 10))
plt.title("image")
plt.subplot(1, 2, 1)
plt.imshow(new_img[:, :, H], cmap="gray")
# plt.imshow(new_img[:, :, H])
plt.subplot(1, 2, 2)
plt.imshow(image[:, :, H], cmap="gray")

In [ ]:
new_img[:, :, H]

In [ ]:
test_data['image'].shape

In [ ]:
label[:, :, H]

In [ ]:
image[:, :, H]

In [ ]:
test_data.keys()

In [ ]:
test_data['image_meta_dict']